In [27]:
import geopandas as gpd
import rasterio

In [12]:
#note you can get this at GA web services https://gaservices.ga.gov.au/site_1/rest/services/GA_Surface_Geology/MapServer/10
lith = gpd.read_file(r'AUS_GA_1M_GUPoly_Lithology.geojson')
faults = gpd.read_file(r'AUS_GA_1M_Faults.geojson')

<b>Investigate for broad scale, rolled up or any useful metrics

In [5]:
lith.columns

Index(['objectid', 'mapsymbol', 'plotsymbol', 'stratno', 'name', 'descr',
       'typename', 'type_uri', 'geolhist', 'repage_uri', 'yngage_uri',
       'oldage_uri', 'lithology', 'replith_uri', 'morphology', 'obsmethod',
       'confidence', 'source', 'metadata', 'frame', 'resscale', 'captdate',
       'moddate', 'plotrank', 'featureid', 'geolunitid', 'st_area(shape)',
       'st_perimeter(shape)', 'geometry'],
      dtype='object')

In [6]:
lith.obsmethod.value_counts()

synthesis of multiple published descriptions    242700
Name: obsmethod, dtype: int64

In [7]:
lith.confidence.value_counts()

Series([], Name: confidence, dtype: int64)

In [19]:
lith.frame.value_counts()

land surface    242700
Name: frame, dtype: int64

In [16]:
lith.metadata.value_counts()

http://www.ga.gov.au/metadata-gateway/metadata/record/74619/    242700
Name: metadata, dtype: int64

In [13]:
faults.columns

Index(['objectid', 'typename', 'type_uri', 'fltname', 'descr', 'exposure',
       'faultfill', 'defrmstyle', 'defrm_uri', 'mvttype', 'mvttyp_uri',
       'mvtsense', 'dsplcmnt', 'dip', 'dipdirn', 'width', 'geolhist',
       'repage_uri', 'yngage_uri', 'oldage_uri', 'fltsys', 'fltsysid',
       'obsmethod', 'confidence', 'posacc_m', 'source', 'metadata', 'frame',
       'resscale', 'captscale', 'captdate', 'moddate', 'plotrank', 'featcode',
       'featureid', 'faultid', 'st_length(shape)', 'geometry'],
      dtype='object')

In [20]:
lith.replith_uri.value_counts()

http://resource.geosciml.org/classifier/cgi/lithology/material_formed_in_surficial_environment    99376
http://resource.geosciml.org/classifier/cgi/lithology/clastic_sedimentary_rock                    54845
http://resource.geosciml.org/classifier/cgi/lithology/granitoid                                   27558
http://resource.geosciml.org/classifier/cgi/lithology/metamorphic_rock                            15967
http://resource.geosciml.org/classifier/cgi/lithology/basalt                                       8447
http://resource.geosciml.org/classifier/cgi/lithology/carbonate_sedimentary_rock                   7436
http://resource.geosciml.org/classifier/cgi/lithology/clastic_mudstone                             6412
http://resource.geosciml.org/classifier/cgi/lithology/rhyolitoid                                   6251
http://resource.geosciml.org/classifier/cgi/lithology/sedimentary_rock                             5324
http://resource.geosciml.org/classifier/cgi/lithology/gabbroic_r

In [15]:
faults.posacc_m.value_counts()

250     77147
100     15215
500      6243
1000     4530
50        734
25         83
Name: posacc_m, dtype: int64

In [22]:
faults.obsmethod.value_counts()

synthesis of multiple published descriptions    103952
Name: obsmethod, dtype: int64

In [23]:
faults.confidence.value_counts()

Series([], Name: confidence, dtype: int64)

In [29]:
#make a faults by positional accuracy raster
from rasterio.features import rasterize
col = 'posacc_m'
gdf = faults
with rasterio.open(r'model_raster.tif') as src:
    print(src.bounds)
    print(src.meta)
    shapes = ((geom,value) for geom, value in zip(gdf.geometry, gdf[col]))
    
    varray = rasterize(shapes,
                   out_shape=(src.height, src.width),
                   transform=src.transform,
                   all_touched=True,
                   dtype='float32')
    
    with rasterio.open('faults_1M.tif', 'w', **src.meta) as dst:
        dst.write(varray, indexes=1)


BoundingBox(left=113.0, bottom=-44.0, right=154.0, top=-10.0)
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 5087, 'height': 4710, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.008059760172989974, 0.0, 113.0,
       0.0, -0.007218683651804671, -10.0)}
